**Summary**

DragonNet.
    
    - Reference: Shi, C., Blei, D. M., & Veitch, V. (2019). Adapting neural networks for the estimation of treatment effects. arXiv preprint arXiv:1906.02120.
    
    - Link: https://github.com/claudiashi57/dragonnet

### Libraries & Parameters

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Basic libraries
import random
import time
import numpy  as np
import pandas as pd
from   os     import listdir
# Sklearn library
from sklearn.preprocessing   import StandardScaler
# Tensorflow library
import tensorflow                  as tf
from   tensorflow.keras.optimizers import Adam, SGD
from   tensorflow.keras.callbacks  import TerminateOnNaN
from   tensorflow.keras.callbacks  import EarlyStopping
from   tensorflow.keras.callbacks  import ReduceLROnPlateau
# from   tensorflow.keras.utils      import plot_model
# User libraries
from utils.data_loading import Synthetic_dataset, TWINS_dataset, IHDP_dataset, ACIC_dataset
from utils.metrics      import PEHE, ATE
from utils.Loss         import *
from utils.DragonNet    import *
print('[INFO] All libraries were imported')



# Random generators initialization
seed=42
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
print('[INFO] Random generators were initialized')

In [ ]:
targeted_regularization    = True # {True, False}

output_dir                 = ''
knob_loss                  = dragonnet_loss_binarycross
loss_ratio                 = 1.
validation_split           = 0.2
batch_size                 = 8
verbose                    = False


metrics = [regression_loss, binary_classification_loss, treatment_accuracy, track_epsilon]

if targeted_regularization:
    loss = make_tarreg_loss(ratio=loss_ratio, dragonnet_loss=knob_loss)
else:
    loss = knob_loss




problem = "Synthetic" # {"IHDP", "Synthetic", "TWINS", "ACIC"}
path = "Data/Synthetic/" # {"Data/Synthetic/", "Data/IHDP/", "Data/TWINS/", "Data/ACIC/"}
filename = f"./Results/{problem}_DragonNet.csv"


if "Synthetic" in problem:
    DataLoader = Synthetic_dataset(path=path)
elif "IHDP" in problem:
    DataLoader = IHDP_dataset(path=path)
elif "TWINS" in problem:
    DataLoader = TWINS_dataset(path=path)
elif "ACIC" in problem:
    DataLoader = ACIC_dataset(path=path, train_size=0.8, random_state=1983)  

### Simulation loop

In [ ]:
results = {'ATE': [], 'Error_ATE': [], 'Error_PEHE':[]}
for idx in range(DataLoader.nProblems):

    # Start timer
    #
    start1 = time.time()
    
    
    # Load training data
    trainX, trainT, trainY, train_potential_Y = DataLoader.getTraining( idx )

    # Load testing data
    testX, testT, testY, test_potential_Y  = DataLoader.getTesting( idx )
    #
    print('Simulation: ', idx)
    print('[INFO] Dataset imported')
    print('[INFO] Number of training instances: ', trainX.shape[0])
    
 
    
    
    # Setup scaler for inputs
    #
    scalerX = StandardScaler()
    trainX  = scalerX.fit_transform( trainX )
    testX   = scalerX.transform( testX )
    
    

    # Setup scaler for target variable
    #
    scalerY       = StandardScaler()
    trainY_scaled = scalerY.fit_transform( trainY.reshape(-1,1) ).squeeze(-1)    
    
    # Setup DragonNet
    #
    dragonnet = make_dragonnet(trainX.shape[1], 0.01)
    
    
    # Create outputs for DragonNet (concatenate Y & T)
    #
    yt_train = np.concatenate([trainY_scaled.reshape(-1,1), trainT.reshape(-1,1)], axis = 1)
    
    
    #
    #
    # *** Training - Phase I ***
    #
    #
    
    # Compile network
    #
    dragonnet.compile(optimizer = Adam(lr=1e-3), 
                      loss      = loss, 
                      metrics   = metrics)

    # Setup callbacks
    callbacks = [TerminateOnNaN(),
                 EarlyStopping(monitor   = 'val_loss', 
                               patience  = 2, 
                               min_delta = 0.),
                 ReduceLROnPlateau(monitor   = 'loss', 
                                   factor    = 0.5, 
                                   patience  = 5, 
                                   verbose   = verbose, 
                                   mode      = 'auto', 
                                   min_delta = 1e-8, 
                                   cooldown  = 0, 
                                   min_lr    = 0)]

    start_time = time.time()

    # Training
    #
    dragonnet.fit(trainX, yt_train, 
                  callbacks        = callbacks,
                  validation_split = validation_split,
                  epochs           = 100,
                  batch_size       = batch_size, 
                  verbose          = verbose)


    print("[INFO] Training - Phase I - Time %.2f secs" % (time.time() - start_time) )

    
    
    
    
    
    
    #
    #
    # *** Training - Phase II ***
    #
    #    

    # Setup callbacks
    #
    callbacks = [TerminateOnNaN(),
                 EarlyStopping(monitor   = 'val_loss', 
                               patience  = 40, 
                               min_delta = 0.),
                 ReduceLROnPlateau(monitor   = 'loss', 
                                   factor    = 0.5, 
                                   patience  = 5, 
                                   verbose   = verbose, 
                                   mode      = 'auto',
                                   min_delta = 0., 
                                   cooldown  = 0, 
                                   min_lr    = 0)
    ]

    # Compile network
    #
    dragonnet.compile(optimizer = SGD(lr=1e-5, momentum=0.9, nesterov=True), 
                      loss      = loss,
                      metrics   = metrics)




    start_time = time.time()

    # Training
    #
    dragonnet.fit(trainX, yt_train, 
                  callbacks        = callbacks,
                  validation_split = validation_split,
                  epochs           = 300,
                  batch_size       = batch_size, 
                  verbose          = verbose)

    print("[INFO] Training - Phase II - Time %.2f secs" % (time.time() - start_time) )


    
    
    
    #
    #
    # *** Predictions ***
    #
    #       
    print("[INFO] Model evaluation")
    yt_hat_test  = dragonnet.predict(testX, verbose=False)

    # Get predictions
    #
    test_y_hat = yt_hat_test[:,:2]


    # Apply inverse transformation
    #
    test_y_hat[:,0] = scalerY.inverse_transform( test_y_hat[:,0].reshape(-1,1) ).squeeze()
    test_y_hat[:,1] = scalerY.inverse_transform( test_y_hat[:,1].reshape(-1,1) ).squeeze()
    
    
    # Get propensity score
    #
    propensity_score = yt_hat_test[:,2]
   
    

    
    
    
    # ATE
    #
    real_ATE = ( test_potential_Y[:,1] - test_potential_Y[:,0] ).mean()
    
    
    # Error PEHE
    #
    Error_PEHE = PEHE(test_potential_Y, test_y_hat)
    
    
    # Error ATE
    #
    Error_ATE = ATE(test_potential_Y, test_y_hat)  
    
        
    # Store errors of PEHE and ATE
    #
    results['ATE']            += [ np.round(real_ATE,   6) ]
    results['Error_ATE']      += [ np.round(Error_ATE,  6) ]
    results['Error_PEHE']     += [ np.round(Error_PEHE, 6) ]

    print('[INFO] Time %.2f seconds\n\n' % (time.time() - start1))
    


    
    # Save results (at each iteration)
    df = pd.DataFrame( results )
    df['Problem'] = [f"{problem} {x}" for x in df.index]
    df.to_csv(filename, index=False)